This notebook is a tutorial for running PET.
From the benchmark, we identified the optimal setting for running PET, which consists of 3 steps:
1. perform differential expression analysis with DESeq2
2. perform pathway analysis with underlying methods
3. combine the result using PET

File required:
1. Data matrix, raw read count strongly recommended.
2. Pathway file, in gmt format.

Example data:\
We provided an example read count matrix text file and gmt pathway file (KEGG) as an example.

Notes:\
Though template script and command for running GSEA and DESeq2 are provided, users should feel free to use any differential expression analysis methods and GSEA mode as wanted, as long as the output format is consistent for correctly parsing the results.

In [1]:
from fisher_test import run_fisher_test
from enrichr import run_enrichr
from helper import *
import os, sys
import numpy as np

In [2]:
# create result directory
out_dir = 'example/'
create_dir(out_dir)

Step 1. 
Perform DEseq2 analysis, the template DESeq2 script is provided in deseq2_template.R.\
Note: **Raw read count** is required for DESeq2, first column as gene name, rest column as group1+group2 expression in order.

In [4]:
# helper function to format DEseq2 script, please specify the result file with .csv suffix
format_deseq2_script(read_count_file_path='example/example_data.txt', sample_num=62, group1_id='poor_survival',
                     group2_id='better_survival', group1_num=33, group2_num=29, 
                     result_file_path='example/deseq_result.csv', script_name='deseq_analysis.R')

DESeq2 script written to deseq_analysis.R


In [11]:
# run DESeq2 script
! Rscript deseq_analysis.R

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min


Attaching package: ‘S4Vectors’

The followin

class: DESeqDataSet 
dim: 53657 62 
metadata(1): version
assays(1): counts
rownames(53657): TSPAN6 TNMD ... LINC01144 AC007389.5
rowData names(0):
colnames(62): TCGA_AA_3850.Tumor.Rep244 TCGA_AA_3845.Tumor.Rep208 ...
  TCGA_DM_A1HA.Tumor.Rep410 TCGA_DM_A1HB.Tumor.Rep417
colData names(1): condition
estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing
-- replacing outliers and refitting for 5411 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)
estimating dispersions
fitting model and testing


In [6]:
# extract DESeq2 result, CSV file only, to .rnk file for GSEA, please do check the direction of DESeq2 result (log2FoldChange column)
# if log2FoldChange>=0 means up-regulation, keep direction=1; otherwise, pass direction=-1
generate_rank_file(deseq_result_file='example/deseq_result.csv', out_file='example/prerank.rnk', direction=1)

In [6]:
# To run GSEA, please download GSEA command line tool from here: http://www.gsea-msigdb.org/gsea/downloads.jsp
# please change the gsea-cli.sh pathway as needed
! ~/GSEA_test/GSEA_cmd/gsea-cli.sh GSEAPreranked -gmx example/c2.cp.kegg.v2023.1.Hs.symbols.gmt -collapse No_Collapse -mode Max_probe -norm meandiv -nperm 1000 -rnk example/prerank.rnk  -scoring_scheme weighted -rpt_label example_test   -create_svgs false -include_only_symbols true -make_sets true -plot_top_x 5 -rnd_seed timestamp -set_max 500 -set_min 15 -zip_report false -out example/gsea_out_example/  

Using system JDK.
494      [INFO  ] - Parameters passing to GSEAPreranked.main:
496      [INFO  ] - rnk	example/prerank.rnk
496      [INFO  ] - gmx	example/c2.cp.kegg.v2023.1.Hs.symbols.gmt
496      [INFO  ] - rpt_label	example_test
496      [INFO  ] - collapse	No_Collapse
496      [INFO  ] - zip_report	false
496      [INFO  ] - gui	false
496      [INFO  ] - out	example/gsea_out_example/
496      [INFO  ] - mode	Max_probe
496      [INFO  ] - norm	meandiv
496      [INFO  ] - nperm	1000
496      [INFO  ] - scoring_scheme	weighted
496      [INFO  ] - include_only_symbols	true
496      [INFO  ] - make_sets	true
496      [INFO  ] - plot_top_x	5
496      [INFO  ] - rnd_seed	timestamp
496      [INFO  ] - create_svgs	false
496      [INFO  ] - set_max	500
497      [INFO  ] - set_min	15
638      [INFO  ] - Made Vdb dir JIT: /Users/luopin/GSEA_test/ENCODE_RNA-seq/PET/mar10
656      [INFO  ] - Begun importing: RankedList from: example/prerank.rnk
1130     [INFO  ] - Your current version of GSEA is

GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
shuffleGeneSet for GeneSet 91/176 nperm: 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
shuffleGeneSet for GeneSet 96/176 nperm: 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorted_scored: 501 / 1000
shuffleGeneSet for GeneSet 101/176 nperm: 1000
GeneSetCohorted: 501 / 1000
GeneSetCohorte

Before running other methods, we shall prune the pathways, which will drop pathways with gene_num > max_num and gene_num < min_num and also remove any gene that's not present in the expression matrix.

In [3]:
# please keep the gene number setting same as GSEA command
pruned_pathway_dict, gene_universe = prune_gmt(file_name='example/c2.cp.kegg.v2023.1.Hs.symbols.gmt', 
                                        out_file_name='example/c2.cp.kegg.v2023.1.Hs.symbols.cleaned.gmt', 
                                        expr_matrix_file='example/example_data.txt', 
                                        min_gene_num=15, max_gene_num=500)

53657 genes in the expression matrix
KEGG_GLYCOSPHINGOLIPID_BIOSYNTHESIS_GLOBO_SERIES removed due to gene number
KEGG_NON_HOMOLOGOUS_END_JOINING removed due to gene number
KEGG_CIRCADIAN_RHYTHM_MAMMAL removed due to gene number
KEGG_VALINE_LEUCINE_AND_ISOLEUCINE_BIOSYNTHESIS removed due to gene number
KEGG_TAURINE_AND_HYPOTAURINE_METABOLISM removed due to gene number
KEGG_FOLATE_BIOSYNTHESIS removed due to gene number
KEGG_LIMONENE_AND_PINENE_DEGRADATION removed due to gene number
KEGG_SULFUR_METABOLISM removed due to gene number
KEGG_RIBOSOME removed due to gene number
KEGG_PROTEASOME removed due to gene number


Fisher test takes a set gene of interest and a pathway file (dict only here). We'll perform this step for both top up and down-regulated genes, sorted by DESeq2 result p-value.

In [4]:
# for extracting the top DEGs, we'll simply query the rank file here
up_regulated_gene_set = extract_top_gene(rank_file='example/prerank.rnk', num_gene=200, direction=1)
down_regulated_gene_set = extract_top_gene(rank_file='example/prerank.rnk', num_gene=200, direction=-1)

In [6]:
# run fisher test for both sets of genes against the pathway file
run_fisher_test(pathway_dict=pruned_pathway_dict, gene_set=up_regulated_gene_set, 
                gene_universe_num=len(gene_universe), out_file_name='example/fisher_up.txt')
run_fisher_test(pathway_dict=pruned_pathway_dict, gene_set=down_regulated_gene_set, 
                gene_universe_num=len(gene_universe), out_file_name='example/fisher_down.txt')

Start performing fisher test!
KEGG_N_GLYCAN_BIOSYNTHESIS
KEGG_OTHER_GLYCAN_DEGRADATION
KEGG_O_GLYCAN_BIOSYNTHESIS
KEGG_GLYCOSAMINOGLYCAN_DEGRADATION
KEGG_GLYCOSAMINOGLYCAN_BIOSYNTHESIS_KERATAN_SULFATE
KEGG_GLYCEROLIPID_METABOLISM
KEGG_GLYCOSYLPHOSPHATIDYLINOSITOL_GPI_ANCHOR_BIOSYNTHESIS
KEGG_GLYCEROPHOSPHOLIPID_METABOLISM
KEGG_ETHER_LIPID_METABOLISM
KEGG_ARACHIDONIC_ACID_METABOLISM
KEGG_LINOLEIC_ACID_METABOLISM
KEGG_ALPHA_LINOLENIC_ACID_METABOLISM
KEGG_SPHINGOLIPID_METABOLISM
KEGG_GLYCOSPHINGOLIPID_BIOSYNTHESIS_LACTO_AND_NEOLACTO_SERIES
KEGG_GLYCOSPHINGOLIPID_BIOSYNTHESIS_GANGLIO_SERIES
KEGG_RIBOFLAVIN_METABOLISM
KEGG_NICOTINATE_AND_NICOTINAMIDE_METABOLISM
KEGG_PANTOTHENATE_AND_COA_BIOSYNTHESIS
KEGG_AMINOACYL_TRNA_BIOSYNTHESIS
KEGG_BASAL_TRANSCRIPTION_FACTORS
KEGG_SNARE_INTERACTIONS_IN_VESICULAR_TRANSPORT
KEGG_LYSOSOME
KEGG_CARDIAC_MUSCLE_CONTRACTION
KEGG_RENIN_ANGIOTENSIN_SYSTEM
KEGG_TASTE_TRANSDUCTION
KEGG_PROXIMAL_TUBULE_BICARBONATE_RECLAMATION
KEGG_PATHOGENIC_ESCHERICHIA_COLI_INFEC

Running enrichr is similar to run fisher test, which requires a set of genes of interest and a pathway file.\
To caculate the final enrichment score, enrichr requires one step of permutation, **we recommend to set perm_num to 1000**.\
If the permutation file already exists, we'll use the existing one; if not, a new permutation will be performed. **NOTE**: the permutaiton step might take some time.

In [7]:
run_enrichr(pathway_dict=pruned_pathway_dict, gene_set=up_regulated_gene_set,
            gene_universe=gene_universe, out_file_name='example/enrichr_up.txt', 
            permutation_num=1000, permutation_file_name='example/enrichr_kegg_permutation_1000.txt')
run_enrichr(pathway_dict=pruned_pathway_dict, gene_set=down_regulated_gene_set,
            gene_universe=gene_universe, out_file_name='example/enrichr_down.txt', 
            permutation_num=1000, permutation_file_name='example/enrichr_kegg_permutation_1000.txt')

example/enrichr_kegg_permutation_1000.txt does not exist, initializing permutation
Permutation 0
Permutation 50
Permutation 100
Permutation 150
Permutation 200
Permutation 250
Permutation 300
Permutation 350
Permutation 400
Permutation 450
Permutation 500
Permutation 550
Permutation 600
Permutation 650
Permutation 700
Permutation 750
Permutation 800
Permutation 850
Permutation 900
Permutation 950
Permutation result written to  example/enrichr_kegg_permutation_1000.txt
Performing fisher test now...
Results written to  example/enrichr_up.txt
********************
example/enrichr_kegg_permutation_1000.txt already exists, reading file now...
Performing fisher test now...
Results written to  example/enrichr_down.txt
********************


After getting results from three underlying methods, the last step is to run PET. We'll perform this step for both up and down-regulated genes. 